In [1]:
!pip install mp_api -q
!pip install --pre bibtexparser -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.0 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
spacy 3.4.0 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.6.4 which is incompatible.
gradient-utils 0.5.0 requires pymongo<4.0.0,>=3.11.0, but you have pymongo 4.6.2 which is incompatible.


In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append('/notebooks/ranker/src/')

from get_from_mp_api import *

# Start from read-in point below, just above SC section 

# Read in all materials and take only flat ones

In [ ]:
df = pd.read_csv("csv_data/material_flatness_score_data/Compound_flatness_score_segments.csv")

In [ ]:
df.columns

In [ ]:
df_slice = df[['MP-ID','Flat/not-flat','Flat segments']]
# extract materials with at least one flat band across the whole k-space path chosen
df_flat_only = df_slice.loc[df_slice['Flat/not-flat']==True]
df_flat_only

# Add synthNN scores

In [ ]:
df_synthNN_scores = pd.read_csv("csv_data/SynthNN_final_scores.csv")
df_synthNN_scores

In [ ]:
df_flat_only['formula']= df_synthNN_scores['Formula'].values
df_flat_only['synth_score']= df_synthNN_scores['SynthNN_score'].values
df_flat_only

# Add sublattice elements 
### **Note at this point, we cut out rows from the 33399 array to match the smaller 33251 array which was actually clustered
### When adding e.g. CNN_synth scores, we will need to add it above, before this cut is made, so that all rows match up

In [ ]:
df_sublattices = pd.read_csv("csv_data/material_flatness_score_data/Flat_band_sublattices.csv")
df_sublattices.rename(columns={'MP_index': 'MP-ID'}, inplace=True)
df_sublattices.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
temp_merged_df = pd.merge(df_flat_only, df_sublattices, on='MP-ID', how='inner')
temp_merged_df.rename(columns={'MP-ID': "material_id", "Flat/not-flat":"flat", "Flat segments": "flat_segments"}, inplace=True)
temp_merged_df.set_index("material_id", inplace=True)
temp_merged_df.head()

# Add materials project information

In [ ]:
merged_df = temp_merged_df
material_ids = merged_df.index.to_list()
merged_df["paper_titles"] = np.nan
merged_df["experimentally_observed"] = np.nan
merged_df["band_gap"] = np.nan



titles, titles_rejected = get_titles(material_ids)
experimentally_observed = get_experimentally_observed(material_ids)
band_gap = get_band_gap(material_ids)


merged_df["paper_titles"].update(titles)
merged_df["experimentally_observed"].update(experimentally_observed)
merged_df["band_gap"].update(band_gap)


In [ ]:
merged_df.to_csv("temp_in_case_this_takes_ages.csv")

In [3]:
def convert_to_list(string):
    if string.startswith('[') and string.endswith(']'):
        return eval(string)
    else:
        return string

main_df = pd.read_csv("temp_in_case_this_takes_ages.csv", converters={'paper_titles': convert_to_list})
main_df.set_index("material_id", inplace=True)
main_df.head()

,flat,flat_segments,formula,synth_score,Cluster_ID,cluster_size,sublattice_element,dbcv_cluster,probabilities,paper_titles,experimentally_observed,band_gap
material_id,,,,,,,,,,,,
mp-7,True,[4],S,0.013924,-1,24875,S,0.846820,0.000000,"[The lattice of rhombohedral sulfur, The cryst...",True,2.2079
mp-25,True,[3],N2,0.127548,63,282,N,0.212178,0.104165,[Electron Diffraction and the Structure of alp...,True,7.3685
mp-35,True,"[1, 2, 3, 4]",Mn,0.007099,-1,24875,Mn,0.846820,0.000000,"[Crystal structure of alpha-manganese, Antifer...",True,0.0000
mp-43,True,"[1, 2, 3, 4]",U,0.001247,-1,24875,U,0.846820,0.000000,[The crystal structure of the beta phase of ur...,True,0.0000
mp-83,True,"[3, 4]",S,0.013924,38,22,S,0.468464,0.972129,[X-Ray Structure Analyses of Two Allotropes of...,True,2.5657


# Remove Banned Titles (usually refering to materials project, high-throughput methods, ect.)

In [4]:
banned_words = ["data base", "database", "throughput"]

def remove_banned_titles(titles):
    allowed_titles = []
    for title in titles:
        allowed = True
        for banned_word in banned_words:
            if banned_word in title.lower():
                allowed = False
                break
        if allowed:
            allowed_titles.append(title)
    return allowed_titles

main_df.paper_titles = main_df["paper_titles"].apply(remove_banned_titles)

# Add Synth Scores

In [5]:
xie_spp_scores = pd.read_csv("/notebooks/ranker/notebooks/Create_dataframe/csv_data/flat_material_sythn_scores.csv", index_col="material_id")
# xie_spp_scores
main_df["synth_score_2"] = np.nan
main_df["synth_score_2"].update(xie_spp_scores.score)
main_df["synth_score_2"]

material_id
mp-7          0.999939
mp-25         0.893845
mp-35         0.999977
mp-43         1.000000
mp-83         0.999928
                ...   
mp-1443188    0.982184
mp-1443279    0.990963
mp-1443442    0.992047
mp-1445771    0.515956
mp-1475733    0.998813
Name: synth_score_2, Length: 33251, dtype: float64

In [6]:
main_df

,flat,flat_segments,formula,synth_score,Cluster_ID,cluster_size,sublattice_element,dbcv_cluster,probabilities,paper_titles,experimentally_observed,band_gap,synth_score_2
material_id,,,,,,,,,,,,,
mp-7,True,[4],S,0.013924,-1,24875,S,0.846820,0.000000,"[The lattice of rhombohedral sulfur, The cryst...",True,2.2079,0.999939
mp-25,True,[3],N2,0.127548,63,282,N,0.212178,0.104165,[Electron Diffraction and the Structure of alp...,True,7.3685,0.893845
mp-35,True,"[1, 2, 3, 4]",Mn,0.007099,-1,24875,Mn,0.846820,0.000000,"[Crystal structure of alpha-manganese, Antifer...",True,0.0000,0.999977
mp-43,True,"[1, 2, 3, 4]",U,0.001247,-1,24875,U,0.846820,0.000000,[The crystal structure of the beta phase of ur...,True,0.0000,1.000000
mp-83,True,"[3, 4]",S,0.013924,38,22,S,0.468464,0.972129,[X-Ray Structure Analyses of Two Allotropes of...,True,2.5657,0.999928
...,...,...,...,...,...,...,...,...,...,...,...,...,...
mp-1443188,True,[4],ZnCuF4,0.574649,-1,24875,Cu,0.846820,0.000000,[],False,0.6531,0.982184
mp-1443279,True,"[1, 3, 4]",Cr(OF)3,0.035679,-1,24875,F,0.846820,0.000000,[],False,0.4652,0.990963
mp-1443442,True,"[3, 4]",VF5,0.727455,-1,24875,F,0.846820,0.000000,[],False,2.5453,0.992047


In [7]:
main_df["sublattice_element"].loc['mp-25']

'N'

# Add structure information of flat-band sublattice

In [7]:
temp_df_2 = main_df
material_ids = main_df.index.to_list()
sub_elems = main_df["sublattice_element"]
temp_df_2["sub_lattice_structure"] = np.nan

sub_structs = get_sub_structure(material_ids, sub_elems)

temp_df_2["sub_lattice_structure"].update(sub_structs)

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/3251 [00:00<?, ?it/s]

### Save

In [8]:
temp_df_2.to_csv("df_full_flat_mat_info.csv")

# Add charge density grid and scalar field

In [4]:
main_df = pd.read_csv("df_full_flat_mat_info.csv", converters={'paper_titles': convert_to_list})
main_df.set_index("material_id", inplace=True)
main_df.head()

,flat,flat_segments,formula,synth_score,Cluster_ID,cluster_size,sublattice_element,dbcv_cluster,probabilities,paper_titles,experimentally_observed,band_gap,synth_score_2,sub_lattice_structure
material_id,,,,,,,,,,,,,,
mp-7,True,[4],S,0.013924,-1,24875,S,0.846820,0.000000,"[The lattice of rhombohedral sulfur, The cryst...",True,2.2079,0.999939,"{'@module': 'pymatgen.core.structure', '@class..."
mp-25,True,[3],N2,0.127548,63,282,N,0.212178,0.104165,[Electron Diffraction and the Structure of alp...,True,7.3685,0.893845,"{'@module': 'pymatgen.core.structure', '@class..."
mp-35,True,"[1, 2, 3, 4]",Mn,0.007099,-1,24875,Mn,0.846820,0.000000,"[Crystal structure of alpha-manganese, Antifer...",True,0.0000,0.999977,"{'@module': 'pymatgen.core.structure', '@class..."
mp-43,True,"[1, 2, 3, 4]",U,0.001247,-1,24875,U,0.846820,0.000000,[The crystal structure of the beta phase of ur...,True,0.0000,1.000000,"{'@module': 'pymatgen.core.structure', '@class..."
mp-83,True,"[3, 4]",S,0.013924,38,22,S,0.468464,0.972129,[X-Ray Structure Analyses of Two Allotropes of...,True,2.5657,0.999928,"{'@module': 'pymatgen.core.structure', '@class..."


In [ ]:
# Add full structure information of flat-band sublattice

In [5]:
temp_df_3 = main_df
material_ids = main_df.index.to_list()
temp_df_3["lattice_structure"] = np.nan

structs = get_structure(material_ids)

temp_df_3["lattice_structure"].update(structs)

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/10000 [00:00<?, ?it/s]

Retrieving SummaryDoc documents:   0%|          | 0/3251 [00:00<?, ?it/s]

In [6]:
temp_df_3.to_csv("df_full_structure.csv")